# 데이터셋 일부 선정

<br>

### 전체 : 4730881

In [1]:
import os, glob
import zipfile
import random
from random import shuffle
from itertools import chain

In [2]:
def grep_recur(base_path, pattern="*.*"):
    sub_greps = list(chain(*[grep_recur(dp, pattern) for dp in grep_dirs(base_path)]))
    return grep_files(base_path, pattern) + sub_greps

def grep_files(base_path, pattern="*.*"):
    return glob.glob("{}/{}".format(base_path, pattern))

def grep_dirs(base_path):
    file_paths = [os.path.join(base_path, name) for name in os.listdir(base_path)]
    return [p for p in file_paths if os.path.isdir(p)]

In [3]:
anno_path = "/ds/processed/annotation/"
frame_path = "/ds/processed/vc-one-frame/"

In [4]:
json_paths = grep_recur(anno_path, pattern="*.json")

In [5]:
print(len(json_paths))

4730881


In [10]:
# import json
# with open("/ds/bench/annot_json_list.json", "w+") as f:
#     json.dump(json_paths, f)

In [ ]:
random.seed(860515)

In [ ]:
shuffle(json_paths)

In [ ]:
# num_select = 200000
num_select = len(json_paths)

In [ ]:
selected_json_paths = json_paths[:num_select]
sample_json_paths = selected_json_paths[:100]
print(len(selected_json_paths))
print(len(sample_json_paths))
print(selected_json_paths[0])

In [ ]:
def package_pair(zip_dir_path, js_paths, pack_size=512):
    json_blocks = [js_paths[i:i+pack_size] for i in (range(0, len(js_paths), pack_size))]
    
    for block_idx, js_block in enumerate(json_blocks):
        zip_file_path = "{}/frames-{:05d}.zip".format(zip_dir_path, block_idx)
        package = zipfile.ZipFile(zip_file_path, 'w')
        
        for p in js_block:
            arcname = p.replace("/ds/processed/annotation/", "")
            arcname = arcname.replace("/", "-")
            package.write(p, arcname=arcname, compress_type=zipfile.ZIP_DEFLATED)
        
            frame_path = p.replace("/ds/processed/annotation/", "/ds/processed/vc-one-frame/")
            frame_path = frame_path.replace("json", "jpg")
            arcname = frame_path.replace("/ds/processed/vc-one-frame/", "")
            arcname = arcname.replace("/", "-")
            package.write(frame_path, arcname=arcname, compress_type=zipfile.ZIP_DEFLATED)
            
        print("*** complete package [{}] ***".format(block_idx))

# annotation 및 frame 압축

In [ ]:
# package("/ds/bench", sample_json_paths)
# package("/ds/bench", selected_json_paths)
package_pair("/ds/bench", selected_json_paths)

# legacy

In [ ]:
# def zip_files(zip_file_path, paths):
#     package = zipfile.ZipFile(zip_file_path, 'w')
#     for p in paths:
#         arcname = p.replace("/ds/processed/annotation/", "")
#         arcname = arcname.replace("/", "-")
#         package.write(p, arcname=arcname, compress_type=zipfile.ZIP_DEFLATED)
        
# def archive_frames(zip_file_path, annot_paths):
#     package = zipfile.ZipFile(zip_file_path, 'w')
#     for p in annot_paths:
#         frame_path = p.replace("/ds/processed/annotation/", "/ds/processed/vc-one-frame/")
#         frame_path = frame_path.replace("json", "jpg")
#         arcname = frame_path.replace("/ds/processed/vc-one-frame/", "")
#         arcname = arcname.replace("/", "-")
#         package.write(frame_path, arcname=arcname, compress_type=zipfile.ZIP_DEFLATED)

# def package(zip_dir_path, json_path):
#     zip_files("{}/anno.zip".format(zip_dir_path), json_path)
#     print("*** complete anno ***")
#     archive_frames("{}/frames.zip".format(zip_dir_path), json_path)
#     print("*** complete frame ***")        